# 0. Importing data & cast_list_as_strings

In [1]:
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from sklearn import svm
import xgboost as xgb
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score

import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
from nltk import *
import os
import statistics
import editdistance
import itertools  
import re

In [2]:
cl = sklearn.linear_model.Perceptron()

In [13]:
full_train_df = pd.read_csv('quora_question_pairs/quora_train.csv')


In [4]:
path_data =  os.path.expanduser('~') 

# use this to train and VALIDATE your solution
full_train_df = pd.read_csv(os.path.join(path_data, os.path.join("Datasets", "kaggle_datasets", "quora", "quora_train_data.csv")))

# use this to provide the expected generalization results
# test_df = pd.read_csv("../data/quora_test_data.csv")
test_data = pd.read_csv(os.path.join(path_data, os.path.join("Datasets", "kaggle_datasets", "quora", "quora_test_data.csv")))

C:\Users\jsreu\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# 1. Raw data cleaning and preprocessing 

###  Exercise 1:  `cast_list_as_strings`

Build a function  **`cast_list_as_strings`** that casts each element in the input list to a string.


In [5]:
all_q1 = list(full_train_df["question1"]) + list(full_train_df["question1"])
all_q2 = list(full_train_df["question2"]) + list(full_train_df["question2"])
all_questions = all_q1 + all_q2

In [6]:
import numpy as np
types_ = [type(q).__name__ for q in all_q1]
np.unique(types_)

array(['float', 'str'], dtype='<U5')

In [7]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

In [8]:
q1_train =  cast_list_as_strings(list(full_train_df["question1"]))
q2_train =  cast_list_as_strings(list(full_train_df["question2"]))
#q1_test  =  cast_list_as_strings(list(test_df["question1"]))
#q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [9]:
import numpy as np
types_ = [type(q).__name__ for q in q2_train]
np.unique(types_)

array(['str'], dtype='<U3')


Use all the questions in train and test partitions to build a single list `all_questions` to fit the `count_vectorizer`

In [10]:
all_questions = q1_train + q2_train

In [11]:
df = full_train_df.copy()
df['question1_string'] = cast_list_as_strings(list(df["question1"]))
df['question2_string'] = cast_list_as_strings(list(df["question2"]))

### Remove Punctuation, substituting and lowering
Stolen from https://www.kaggle.com/benjaminkz/quora-question-pairs-data-cleaning

In [12]:
def common_words_transformation_remove_punctuation(text):
    
    text = text.lower()
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"when's", "when is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"there's", "there is", text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)  # 除了上面的特殊情况外，“\'s”只能表示所有格，应替换成“ ”
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " america ", text)
    text = re.sub(r" u s ", " america ", text)
    text = re.sub(r" uk ", " england ", text)
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text)
    text = re.sub(r" ds ", " data science ", text)
    text = re.sub(r" ee ", " electronic engineering ", text)
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iphone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the us", "america", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    
    punctuation="?:!.,;"

    text = "".join([c for c in text if c not in punctuation])
    
    return text

In [13]:
doc = full_train_df['question1'][0]
doc

'What is the step by step guide to invest in share market in india?'

In [14]:
doc = common_words_transformation_remove_punctuation(doc)
doc

'what is the step by step guide to invest in share market in india '

### Removing Stop_Words

In [15]:
def remove_stopwords(doc):
    stops     = set(stopwords.words("english"))
    token_doc = set(word_tokenize(doc))
    token_doc = list(token_doc - stops)
    return " ".join(token_doc)

df['question1_no_stops'] = [remove_stopwords(quest) for quest in df["question1_string"]]
df['question2_no_stops'] = [remove_stopwords(quest) for quest in df["question2_string"]]

In [16]:
print(doc)
doc = remove_stopwords(doc)

what is the step by step guide to invest in share market in india 


### Stemming

In [17]:
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer

porter    = PorterStemmer()
#lancaster = LancasterStemmer()

In [18]:
from nltk.tokenize import sent_tokenize, word_tokenize

def stem(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(porter.stem(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

In [19]:
print(doc)
doc = stem(doc)

market share step invest india guide


### Lemmatization

In [20]:
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()



In [21]:
def lemm(sentence):
    token_words = word_tokenize(sentence)
    sentence_stemmed = []
    for word in token_words:
        sentence_stemmed.append(wordnet_lemmatizer.lemmatize(word))
        sentence_stemmed.append(" ")
    return "".join(sentence_stemmed)

In [22]:
print(doc)
doc = stem(doc)

market share step invest india guid 


### Putting all together

In [23]:
def preprocess(df, save=False):
    df = df.copy()
    df['question1'] = cast_list_as_strings(list(df["question1"]))
    df['question2'] = cast_list_as_strings(list(df["question2"]))
        
    df['question1_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question1"]]
    df['question2_cleaned'] = [common_words_transformation_remove_punctuation(quest) for quest in df["question2"]]
        
    df['question1_no_stops'] = [remove_stopwords(quest) for quest in df["question1_cleaned"]]
    df['question2_no_stops'] = [remove_stopwords(quest) for quest in df["question2_cleaned"]]
    
    df['question1_Porter_stemmed'] = [stem(quest) for quest in df["question1_no_stops"]]
    df['question2_Porter_stemmed'] = [stem(quest) for quest in df["question2_no_stops"]]
        
    df['question1_Lemmatization']  = [lemm(quest) for quest in df["question1_no_stops"]]
    df['question2_Lemmatization']  = [lemm(quest) for quest in df["question2_no_stops"]]
    
    if save==True:
        df.to_csv('preprocessed_Quora_full_train_data.csv')
        
    return df

In [ ]:
full_train_proc = preprocess(full_train_df, save=True)

In [ ]:
full_train_proc.head()

## 1. CountVectorizer Solution
### Naive CountVectorizer and attempts to improve it

In [ ]:
count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer='word', binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, tokenizer=None, vocabulary=None)

count_vectorizer.fit(all_questions)


In [ ]:
#advanced version with stemming
#this implemenetation failed because stop_words cannot be used if analyzer!='word'

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

lemmatizer = WordNetLemmatizer()
stemmer =  SnowballStemmer(language='english')
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer=stemmed_words, binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, tokenizer=None, vocabulary=None)

count_vectorizer.fit(all_questions)

In [ ]:
#advanced version with lemmatization on top of stemming
#this implemenetation failed because the tokenizer cannot be used if analyzer!='word'
#additionally, if tokenizer!=None token_pattern and stop_words cannot be used

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

lemmatizer = WordNetLemmatizer()
stemmer =  SnowballStemmer(language='english')
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer=stemmed_words, binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 stop_words='english', #I also have a list of more advanced stopwords we could use
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 #token_pattern='(?u)\b\w\w+\b', #not sure why it doesn't work with the default option
                                                                 preprocessor=None, 
                                                                 tokenizer=LemmaTokenizer(), 
                                                                 vocabulary=None)

count_vectorizer.fit(all_questions)

#### Problems with the improvement attempts so far

* if analyzer=stemmed_words --> stop_words and tokenizer cannot be used
* if tokenizer=LemmaTokenizer() --> token_pattern and stop_words cannot be used

To solve these issues, we will not use the analyzer. We tokenize, remove the punctuation and stopwords, apply the regEx, and stem all inside the tokenizer.

#### Final CountVectorizer!

In [ ]:
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import string
import re

lemmatizer = WordNetLemmatizer()
stemmer =  SnowballStemmer(language='english')

stopwords_list=nltk.corpus.stopwords.words('english')
stops = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
stopwords_punctuation = (stopwords_list + list(set(stops) - set(stopwords_list)) + list(string.punctuation))


class Tokenizer(object):
    def __init__(self):
        self.stemmer = stemmer
        self.lemmatizer = lemmatizer
    def __call__(self, articles):
        tokens = re.compile(r'(?u)\b\w\w+\b').findall(articles) 
        tokens = [' '.join(w for w in token.split() if w.lower() not in stopwords_punctuation) for token in tokens]
        tokens_stem = [self.stemmer.stem(t) for t in tokens]
        tokens_lem = [self.lemmatizer.lemmatize(t) for t in tokens_stem]
        return tokens_lem

In [ ]:
count_vectorizer=sklearn.feature_extraction.text.CountVectorizer(analyzer='word', binary=False, decode_error='strict', 
                                                                 encoding = 'utf-8', input='content', lowercase=True, 
                                                                 max_df=1.0, max_features=None, min_df=1,
                                                                 ngram_range=(1,1), #let's play here with different combinations
                                                                 strip_accents='unicode', #we can also try using ascii
                                                                 preprocessor=None, 
                                                                 tokenizer=Tokenizer(), 
                                                                 vocabulary=None)

count_vectorizer.fit(all_questions)

###  Exercise 2:  `get_features_from_df`

Make a function `get_features_from_df` that given a dataframe containing the format of the train data
it returns a scipy sparse matrix with the features from question 1 and question 2 

In [ ]:

def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    
    ############### End exercise ###################

    return X_q1q2

In [ ]:
X_tr_q1q2 = get_features_from_df(train_df,count_vectorizer)
X_te_q1q2  = get_features_from_df(test_df, count_vectorizer)

X_tr_q1q2.shape, train_df.shape, test_df.shape, X_te_q1q2.shape

Now You can use this representation `X_tr_q1q2` to fit a model

In [ ]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear",
                                                   random_state=123)
y_train = train_df["is_duplicate"].values
logistic.fit(X_tr_q1q2, y_train)

In [ ]:
len(mistake_indices) 

In [ ]:
predictions = logistic.predict(X_tr_q1q2)
logistic.score(X_tr_q1q2, y_train)

In [ ]:
def get_mistakes(clf, X_q1q2, y):

    ############### Begin exercise ###################
    predictions = clf.predict(X_q1q2)
    incorrect_predictions = predictions != y 
    incorrect_indices,  = np.where(incorrect_predictions)
    
    ############### End exercise ###################
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions

### Examine mistakes


#####  Exercise 3:  `get_mistakes`

Make a function `get_mistakes` that given a model `clf` a dataframe `df`, the features `X_q1q2` and the target labels `y`returns 


- incorrect_indices: coordinates where the model made a mistake
- predictions: predictions made by the model


In [ ]:
y_train = train_df["is_duplicate"].values

mistake_indices, predictions = get_mistakes(logistic,
                                            X_tr_q1q2, 
                                            y_train)

In [ ]:
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])

In [ ]:
print_mistake_k(89395, mistake_indices, predictions)

# 2. Other Feature Vectors

In [ ]:
#Convert all in string
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    #assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings

#Removing Stops Words
#from nltk.corpus import stopwords
#nltk.download('stopwords')

def remove_stopwords(doc):
    stops     = set(stopwords.words("english"))
    token_doc = set(word_tokenize(doc))
    token_doc = list(token_doc - stops)
    return " ".join(token_doc)

#Adding len Words in common feat
def len_common(q1, q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1.intersection(q2))

#Adding len common words in common feat
def len_not_common(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    return len(q1 ^ q2)

#Adding mean distance between common words 
def mean_dist_not_com(q1,q2):
    q1 = set(word_tokenize(q1)) ; q2 = set(word_tokenize(q2))
    not_comm1 = (q1 ^ q2) - q1
    if len(not_comm1)==0 : not_comm1={''}
    not_comm2 = (q1 ^ q2) - q2
    if len(not_comm2)==0 : not_comm2={''}
    return statistics.mean([editdistance.eval(i[0],i[1]) for i in itertools.product(not_comm1, not_comm2)])

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString)) 
def both_number(q1,q2):
    return hasNumbers(q1) *  hasNumbers(q2) 

# get average number of words
def average_len(question):
    num_words = len(question.split())
    return len(question)/num_words   

# levenshtein distance (for strings of unequal length)
def levenshtein(q1, q2): 
    #create initial array (two for loops since q1 and q2 can differ in length)
    dist_array = []
    for i in range(len(q1)+1):
        dist_array.append([0]*(len(q2)+1))
        dist_array[i][0] = i
    for j in range(len(q2)+1):
        dist_array[0][j] = j

    dist = [0]*3
    for i in range(1,len(q1)+1):
        for j in range(1,len(q2)+1):
            dist[0] = dist_array[i-1][j-1] if q1[i-1]==q2[j-1] else dist_array[i-1][j-1]+1
            dist[1] = dist_array[i][j-1]+1
            dist[2] = dist_array[i-1][j]+1
            dist_array[i][j]=min(dist)
    
    return dist_array[i][j]

# self implemented jaccard similarity
def jaccard_similarity(q1,q2):
    set1 = set(q1.split()) 
    set2 = set(q2.split())
    same = set1.intersection(set2)
    return float(len(same)) / (len(set1) + len(set2) - len(same))

In [94]:
def Add_features(df, scaler, col1_to_transform, col2_to_transform):
    df = df.copy()

    df['len_q1'] = [len(s) for s in df[col1_to_transform]] 
    df['len_q2'] = [len(s) for s in df[col2_to_transform]]
    df['len_q1'] = scaler.fit_transform(df[['len_q1']])
    df['len_q2'] = scaler.fit_transform(df[['len_q2']])
    
    df['len_common'] = [len_common(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    df['len_common'] = scaler.fit_transform(df[['len_common']])

    df['len_not_common'] = [len_not_common(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    df['len_not_common'] = scaler.fit_transform(df[['len_not_common']])
    
    df['mean_dist_not_com'] = [mean_dist_not_com(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    df['mean_dist_not_com'] = scaler.fit_transform(df[['mean_dist_not_com']])
    
    df['both_number'] = [both_number(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    
    df["avg_len_q1"]= df.question1_string.apply(lambda x: average_len(x))
    df["avg_len_q2"]= df.question2_string.apply(lambda x: average_len(x))
    df['avg_len_q1'] = scaler.fit_transform(df[['avg_len_q1']])
    df['avg_len_q2'] = scaler.fit_transform(df[['avg_len_q2']])
    
    df['levenshtein_distance'] = [levenshtein(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    df['levenshtein_distance'] = scaler.fit_transform(df[['levenshtein_distance']])
    
    df['jaccard'] = [jaccard_similarity(df.loc[i,col1_to_transform],df.loc[i,col2_to_transform]) for i in range(len(df))]
    df['jaccard'] = scaler.fit_transform(df[['jaccard']])

    return df

In [ ]:
scaler = MinMaxScaler()

full_train_proc_plus_feat = Add_features(full_train_proc, scaler, 'question1_no_stops', 'question2_no_stops')